# Content Generation Pipeline
### Briefly: 
### Step 1: Main Content Generation Pipeline
* Use Online Research Agent 
* Generates initial Painpoints, Descriptions, Sources, Links, Ideas, Features
* GPT-3.5-turbo 

### Step 2: Writing Improvement Piepline
* Use Offline Prompting (no agent)
* Improves technical writng of content
* Putting content in desired style, length and rought formatting
* Keeps original link 
* GPT-4-turbo

Follow https://www.youtube.com/watch?v=ogQUlS7CkYA&t=789s

# Step 1: Prepareation & Initilization 

### Imports


In [34]:
# imports
 
from langchain_community.chat_models import ChatPerplexity
from langchain_community.chat_models import ChatPerplexity
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage, AIMessage

from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate, SystemMessagePromptTemplate ,PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.chains.summarize import load_summarize_chain
from langchain.tools import BaseTool
from typing import Type
from langchain_core.prompts import ChatPromptTemplate
import os
from dotenv import load_dotenv
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.prompts import MessagesPlaceholder

from scrapingant_client import ScrapingAntClient
import json
from bs4 import BeautifulSoup
import requests


from datetime import datetime
load_dotenv()



True

### API KEYS

In [35]:
serper_key = os.environ.get("SERPER_API_KEY")
scraping_ant_key = os.environ.get ("SCRAPING_ANT_KEY")

### Models

In [36]:
openai_key =  os.environ.get("OPENAI_API_KEY")
pplx_key =  os.environ.get("PERPLEXITY_API_KEY")


gpt3 = ChatOpenAI(temperature=0, openai_api_key=openai_key,model= "gpt-3.5-turbo-0125") # one way to load a model 
gpt4 = ChatOpenAI(temperature=0, openai_api_key=openai_key,model= "gpt-4-0125-preview") # one way to load a model 
pplx = ChatPerplexity(temperature=0.1, pplx_api_key=pplx_key, model="sonar-medium-online") # or sonar-small-onlinesd 
pplx_sm = ChatPerplexity(temperature=0.5, pplx_api_key=pplx_key, model="sonar-small-online") # or sonar-small-onlinesd 
mixtr  = ChatPerplexity (temperature=0 ,pplx_api_key=pplx_key, model='mistral-7b-instruct')

# Step 2: Main Content Generation Pipeline

### Prompts - System Message

In the current pipeline, p_system_message is used 

In [37]:
# p_system_message =  """ 
# You are a world-class researcher and techincal writer who can do detail research and write content on
#  any topic and produce facts based results using the search and scrape_website .
#    you do not make things up nor hallucinate you will try as hard as possible to gather facts and data to back up the research
 
# Please make sure you complete the objective above with the following rules :


# 0. ouput the links of the sites your searched for
# 1. You should do enough research to gather as much information as possible about the objective
# 2. If there are URL of the relevant links and articles, you will scrape it together more information
# 3. After researching, you should think “ is there any new things I should search and scrape based on the data. 
# I  collected to increase research quality?”if the answer is yes. Continue; but don't do this for more than 3 iterations don’t do this more than 3 iterations
# 4. IMPORTANT: you should not make things up nor hallucinate, you should only write facts and data that you have gathered
# 5. After researching, you should think “is the information too generic and broad? If yes, do more research and be more specific with your output, don't do this for more than 3 iterations 
# 6  After researching, you should think  "is the information similar/same from before? If yes, discard the output and search for different, unique outputs
# 7. you should include all reference data and links to back up your research ONLY IF APPLICABLE (i.e. they cam from your search)

# """

p_system_message = """
  you are a world class researcher on products on the internet 

  who can do detail research and write content on
 any topic and produce facts based results using the search and scrape_website .

 Please make sure you complete the objective above with the following rules :

 1. You should do enough research to gather as much information as possible about the objective
# 2. If there are URL of the relevant links and articles, you will scrape it together more information
# 3. After researching, you should think “ is there any new things I should search and scrape based on the data. 
# I  collected to increase research quality?”if the answer is yes. Continue; but don't do this for more than 3 iterations don’t do this more than 3 iterations
# 4. IMPORTANT: you should not make things up nor hallucinate, you should only write facts and data that you have gathered
# 5. After researching, you should think “is the information too generic and broad? If yes, do more research and be more specific with your output, don't do this for more than 3 iterations 
# 6  After researching, you should think  "is the information similar/same from before? If yes, discard the output and search for different, unique outputs
# 7. you should include all reference data and links to back up your research ONLY IF APPLICABLE (i.e. they cam from your search)
"""


# not used
p_system_message_saas_specific =  """ 
You are a world-class researcher who can do detail research on
 any topic in the SaaS, MicroSaaS and software domain. 
 you produce facts based results using the search and scrape_website .
 you do not make things up nor hallucinate you will try as hard as possible to gather facts and data to back up the research
 
you should search well known forums such as: 
Reddit, Quora, Twitter, LinkedIn,  Facebook, Product Hunt, Capterra, G2, industry-specific forums, Slack communities, Hacker News, GitHub Issues, Stack Overflow, Stack Exchange sites, YouTube comments, Amazon product reviews, niche blog comments. 

Please make sure you complete the objective above with the following rules :
1. You should do enough research to gather as much information as possible about the objective
2. If there are URL of the relevant links and articles, you will scrape it together more information
3. Do not come up with your own URL, if you can't find a source,discard that stream of research
4. After researching, you should think “ is there any new things I should search and scrape based on the data. 
I  collected to increase research quality?”if the answer is yes. Continue; but don't do this for more than 2 iterations don’t do this more than 2 iterations
6. IMPORTANT: you should not make things up nor hallucinate, you should only write facts and data that you have gathered
7. In the final output, you should include all reference data and links to back up your research
8. In the final output, you should include our reference and links to back up your research

"""

t_query_list = """ 
{query}
OUTPUT FORMAT FOR EACH ITEM IDENTIFIED, FOLLOW THE OUTPUT LENGTH STRICTLY (FOLLOW STRICTLY AND OUTPUT NOTHING ELSE):

Product Name:
Price:

IMPORTANT OUTPUT LINK BRO, YOU CANNOT MISS THIS STEP :-> link: <this will be the link for the specific product>



The final output should be 5 (items of) Product name:, Price, Link 
"""



### Building Aegnt

#### 1. Search Tool -> Using Serper API

In [38]:
# Serper
def search (query):
    url = "https://google.serper.dev/search"

    payload = json.dumps({
    "q":query
    })
    headers = {
    'X-API-KEY': serper_key,
    'Content-Type': 'application/json'
    }

    response = requests.request("POST", url, headers=headers, data=payload)

    print(response.text)
    return response.text

### 2. Scraping Tool + Summary Tool


https://github.com/ScrapingAnt/scrapingant-client-python?tab=readme-ov-file#sending-custom-headers

main scrape_website function

In [39]:
def scrape_website (objective: str, url: str): 
    print ("**** Scraping Website **** ")

    client =  ScrapingAntClient(token=scraping_ant_key)

    headers = {
        'Cache-Control': 'no-cache',
        'Content-Type': 'application/json'
    }

    res = client.general_request(
            url=url,
            headers=headers
    )

    if res.status_code == 200: 
        soup = BeautifulSoup (res.content,"html.parser")
        text = soup.get_text()

        # print ("text output:" ,text)
        print ("*** Finished website Scrape  ***")
        
        if len(text) >10000:
            ("*** Long text, entering summary function ***")
            out = summary (objective,text)
            ("*** Summary Finished***")
            return out
        else: 

            ("*** Finishd Scraping Function ***")
            return text
    else:
        print (f"HTTP request failed with status code {res.status_code}")

Summary Function -> map reduce summary

In [40]:
def summary (objective,content):
    text_splitter = RecursiveCharacterTextSplitter(
        separators=['\n\n',"\n"], chunk_size=10000,chunk_overlap=500)
    docs  = text_splitter.create_documents([content])
    print ("** received text inputing into ma_prompt")
    map_prompt = """ 
    Write a summary of the following text for {objective}:
    "{text}"

    *** SUMMARY ***: 
    
    """
    
    map_prompt_template = PromptTemplate ( 
        template=map_prompt, input_variables=["text","objective"]
    )
    # print ("** objective **: ", objective)
    # print ("** map prompt template: ** ",map_prompt_template)

    # print  ("*** building chain ***")
    summary_chain = load_summarize_chain(
        llm=gpt3,
        chain_type='map_reduce',
        map_prompt=map_prompt_template,
        combine_prompt=map_prompt_template,
        verbose =True
    )
    (" *** Invokings summary chain **** ")
    out = summary_chain.invoke ({"input_documents":docs,"objective": objective})
    (" *** Finished summary chain **** ")

    return out


Predefined formatting classes XX NOT USED -> good for documentation

In [41]:
# define input
class ScrapeWebsiteInput (BaseModel):
    objective: str = Field (
        description="the objective $ task that users gives to the agent")
    url: str = Field (description=" the url of the website to be scraped ")

# define function
class ScrapeWebsiteTool (BaseTool): 
    name = "scrape_website"
    description ="""useful when you need to get data from a website url, 
    passing both url and objective to the function;
    DO NOT Make up any URL!the url should only be from the search results""" 
    args_schema: Type[BaseModel] = ScrapeWebsiteInput

    def _run (self,objective: str ,url:str):
        return scrape_website(objective,url)

    def _arun(self,url:str):
        raise NotImplementedError(f"Error in implementing the function : {url}")

### 3. Create Agent Painpoint -> Can Wrap in Function
Note: for the agent_llm gpt 3 seems to invoke the best results so far -> because it actually invokes a search where as pplx online model doesn't.
Still need to test out different models. esp.gpt4
Although gpt3 is lacking in the task in generating painpoint titles

In [42]:
# tool list for the agents to use 

def create_main_content_agent ( memory_llm=gpt3 ):

    tools = [
        Tool (
            name = "Search",
            func= search,
            description = """ 
            useful when you need to answer questions about current
            events,data. You should ask targetted questions
            """
        ),
        ScrapeWebsiteTool(),
    ]
    system_message = SystemMessage (content=p_system_message)

    agent_kwargs = {
        "extract_prompt_messages" : [MessagesPlaceholder(variable_name="memory")],
        "system_message" : system_message
    }

    memory = ConversationBufferMemory (
        memory_key="memory",
        return_messages=True,
        llm=memory_llm,
        max_tokens=1000
    )

    # initialize agent
    agent = initialize_agent (
        tools,
        llm=gpt4,
        agent=AgentType.OPENAI_FUNCTIONS,
        verbose=True,
        agent_kwargs=agent_kwargs,
        memory=memory,
    )
    return agent


### Implmentation 
Looping through Shorter but diverse prompt selection 

In [43]:
### This one is currently used -> can modify ###
query_list = [
""" 
top 5 toners that moisturize my skin from sephora

"""
]



# not used currently

# query_list_0327024 = [
#        """ What are the top 5 Trending tech painpoints of 2024 that can potentially be solved by creatig a new saas product?

# OUTPUT FORMAT FOR EACH ITEM, FOLLOW THE OUTPUT LENGTH STRICTLY (FOLLOW STRICTLY AND OUTPUT NOTHING ELSE):

# painpoint:   (an attention grabbing ovrview of the painpoint , OUTPUT LENGTH 6 to 8 word sentence)
# description: (OUTPUT LENGTH: 4 to 5 sentence (For each problem/complaints/painpoints, craft a  description that captures the core problem and and why its SUCH an annoying/relevant issue. 
# your writing style is, expresive, uplifting. witty,professional, and technical )
# link: (if applicable. If not, output "NA")
# source: the platform where you got the information from
# Idea: an innovative, unique, market relavant,trending, high conversion SaaS idea that directly solves the painpoint above  (OUTPUT LENGTH: 5-8 words)
# Features: a list  of 3-5 (vary this) key features of that idea, make sure to be specific (OUTPUT LENGTH 6-8 words)

# The final output should be 5 (items of) painpoints,descriptions, reference links
#     """ ,  
 
#     """What are 5 challenges individuals in a profession face that can be alleviated with a new platform, currently?
#      OUTPUT FORMAT FOR EACH ITEM, FOLLOW THE OUTPUT LENGTH STRICTLY (FOLLOW STRICTLY AND OUTPUT NOTHING ELSE):
# OUTPUT FORMAT FOR EACH ITEM, FOLLOW THE OUTPUT LENGTH STRICTLY (FOLLOW STRICTLY AND OUTPUT NOTHING ELSE):

# painpoint: an concise ovrview of the painpoint , OUTPUT LENGTH 10 to 15 word sentence,
# description: xxx OUTPUT LENGTH: 4 to 5 sentence (For each problem/complaints/painpoints, craft a  description that captures the core problem and and why its SUCH an annoying/relevant issue. 
# your writing style is Craft witty, engaging pain point descriptions using vivid language, analogies, and humor to highlight)
# reference link: (if applicable. If not, output "NA")
#        """,
 
#     "What are 5 issues teams may encounter that could be addressed with a new tool, by 2025?",
#     "What are 5 obstacles enthusiasts have navigated that might have been simpler with a new service, in the last five years?",
#     "What are 5 problems hobbyists will encounter that can be addressed with a new software, in the near future?",
#     "What are 5 challenges collectors have dealt with that can be resolved with a new system, looking back over the past several years?",
#     "What are 5 issues users currently encounter that can be resolved with a new app, in 2024 and beyond?",
#     "What are 5 challenges learners will likely face that can be mitigated with a new app, by 2025?",
#     "What are 5 obstacles hobbyists have navigated that could have been simpler with a new service, in the last five years?",
#     "What are 5 issues creators face that can be solved with a new platform, currently?"
# ]



Put it in a function -> automating getting painpoints

In [44]:
main_content_temp_list = []
improved_content_temp_list= []

In [45]:
# Main function for getting the pointpoints
def generate_main_content(query_list:list,main_content_temp_list:list,iterations:int,memory_llm=gpt3):
    
    # setting a cap for safety purposes
    if iterations > 20:
        print ("This is too many iterations. returning without running code")
        return
    
    if iterations > len (query_list): 
        print (f"your input iterations is more than the prompts in the query list, only running code for {len(query_list)} iterations")
    l = query_list[:iterations]


    # Create Agent
    agent = create_main_content_agent(memory_llm=memory_llm)


    for i, q in enumerate(l) :

        ## need to document this formatting 
        empty_query = SystemMessagePromptTemplate.from_template(t_query_list)
        filled_query =empty_query.format(query = q)

        out = agent.invoke({"input": str (filled_query)}) # actual call 
        
        print (f'*** {i + 1} iteration finished ***')

        main_content_temp_list.append(out["output"]) 
    
    n = iterations * 5
    print (f"*** Sucessfully generated {n} Content ***")

    # Get today's date in the format YYYY-MM-DD
    today_date = datetime.now().strftime("%Y-%m-%d")

    # Construct the filename with today's date
        # Create directory if it doesn't exist
    dir_name = 'raw_main_content'
    if not os.path.exists(dir_name):
        os.makedirs(dir_name)

    # Define the path for the output file
    file_path = os.path.join(dir_name, f'main_content_{today_date}.txt')

    # Writing the pain points to the text file
    with open(file_path, 'w') as file:
        for content in main_content_temp_list:
            file.write(content + '\n')  # Write each pain point on a new line

    print(f"*** Content saved to '{file_path}' ***")
    return main_content_temp_list

    

In [46]:
generate_main_content(query_list,main_content_temp_list,1)



> Entering new AgentExecutor chain...



Invoking: `Search` with `top 5 toners that moisturize skin from Sephora`


{"searchParameters":{"q":"top 5 toners that moisturize skin from Sephora","type":"search","engine":"google"},"organic":[{"title":"Best Toners in 2024 - Sephora","link":"https://www.sephora.com/beauty/best-toners","snippet":"The Ordinary Glycolic Acid 7% Exfoliating Toner. The Ordinary ; OLEHENRIKSEN Glow2OH 7% AHA Exfoliating Dark Spot Toner Clean at Sephora. OLEHENRIKSEN","position":1},{"title":"Best Toners - Sephora","link":"https://www.sephora.com/shop/facial-toner-skin-toner?ref=filters[Rating]=%224%22-%22inf%22","snippet":"Jart+Ceramidin™ Skin Barrier Serum Toner. 436. $39.00 · LANEIGE - Mini Cream Skin Refillable Toner & Moisturizer with Ceramides and Peptides.","priceRange":"$6 to $55","position":2},{"title":"Facial Toner & Skin Toner - Sephora","link":"https://www.sephora.com/shop/facial-toner-skin-toner","snippet":"Shop face toner and skin toner at Sephora. Refresh, nourish and tone your skin with clar

['Here are the top 5 toners that moisturize skin from Sephora:\n\n1. Product Name: The Ordinary Glycolic Acid 7% Exfoliating Toner\n   Price: $13.00\n   Link: [Sephora - The Ordinary Glycolic Acid 7% Exfoliating Toner](https://www.sephora.com/product/the-ordinary-deciem-glycolic-acid-7-toning-solution-P427419)\n\n2. Product Name: Glow Recipe Watermelon Glow PHA + BHA Pore-Tight Toner\n   Price: $35.00\n   Link: [Sephora - Glow Recipe Watermelon Glow PHA + BHA Pore-Tight Toner](https://www.sephora.com/product/glow-recipe-watermelon-glow-pha-bha-pore-tight-toner-P448958)\n\n3. Product Name: OLEHENRIKSEN Glow2OH 7% AHA Exfoliating Dark Spot Toner\n   Price: $35.00\n   Link: [Sephora - OLEHENRIKSEN Glow2OH 7% AHA Exfoliating Dark Spot Toner](https://www.sephora.com/product/olehenriksen-glow2oh-dark-spot-toner-P433817)\n\n4. Product Name: Fresh Rose & Hyaluronic Acid Deep Hydration Toner\n   Price: $48.00\n   Link: [Sephora - Fresh Rose & Hyaluronic Acid Deep Hydration Toner](https://www.se

## Step 3. Writng Improvment Pipeline (offline) -> Currently Used


### Prompt - System Message




In [47]:
# ## *** This is the one currently in use *** ## 
# p_system_message_improve_writing = """ 
#     You are a As a seasoned technical blog writer, you excel in demystifying complex tech concepts with clarity.

#     Your default writing style is: 
#     - Your content is rich with insights, examples, and a touch of humor, making technology accessible and engaging. 
#     Stay ahead of tech trends, and blend professionalism with personality to educate, inspire, and entertain your audience

#     If the user specifices your WRITING STYLE, output content in their exact WRITING STYLE
    
#     the user will input some data, you are able to write content exactly as they tell you to. and output in the exact format they tell you to. 
#     and output the data in the same length as they tell you to. Below is the data.
#     {input_data}.
#     wait for the users input and write content for them.
# """


# # p_system_message_ideas =  """ 
# # You are a world-class researcher  who can do detail research on
# # any topic by combining the context of {input_data}, as well as searching the internet,
# # and produce facts based results using the search and scrape_website. 
# # you do not make things up nor hallucinate you will try as hard as possible to gather facts and data to back up the research
 
# # Please make sure you complete the objective above with the following rules :

# # 1. You should do enough research to gather as much information as possible about the objective
# # 2. If there are URL of the relevant links and articles, you will scrape it together more information
# # 3. After researching, you should think “is there any new things I should search and scrape based on the data. 
# # I  collected to increase research quality?”if the answer is yes. Continue; but don't do this for more than 3 iterations don’t do this more than 3 iterations
# # 4. IMPORTANT: you should not make things up nor hallucinate, you should only write facts and data that you have gathered

# # """


In [48]:
# def improve_writing_ol(improved_content_temp_list:list, main_content_temp_list: list, llm = gpt4): 
#     for i, pp_batch in enumerate (main_content_temp_list):  # need to add to this 
#         count  = 1 
#         # TODO: run ai
#         # q = """ 
#         #     Improve writing or keep the same, based on your writing style and above data. 
            
#         #     final output: 
#         #     FOR EACH PAINPOINT, you output  the items below
#         #     painpoint:  (improve writing, use less metaphors and be more technical should be 8 to 12 words long)
#         #     description: (improve writing , should be 3 to 4 sentences long)
#         #     link: (UNCHAGED FROM THE ORIGINAL, OUTPUT THE EXACT SAME LINK, UNCHAGED FROM THE ORIGINAL, OUTPUT THE EXACT SAME LINK)
#         #     source: (improve writing: only output the platofrm or site name of which the info is found)
#         #     Idea: (improve writing, should be  8 to 15 words long, clearly states the idea and how it would directly solve the problem)
#         #     Features: (improve writing, FOR EACH FEATURE, should expand a little bit on this feature should be 8-15 words long)
    
#         #     """
#         q =  """
#             OUTPUT FORMAT:
#             - Product Name: xxxx
#             - Price: $xx.xx
#             - Details: xxxxxx
#             - Link: (UNCHAGED FROM THE ORIGINAL, OUTPUT THE EXACT SAME LINK, UNCHAGED FROM THE ORIGINAL, OUTPUT THE EXACT SAME LINK)
#             """
        
#         # initialize chain 
#         t = ChatPromptTemplate.from_messages ([
#             ("system",p_system_message_improve_writing),
#             ("human",q)
#             ])

#         ideas_chain = t| llm
#         out = ideas_chain.invoke({"input_data", pp_batch}) 


#         print (f"*** Improve Writing Batch {i + 1} Complete! ***")

#         # append to ideas_tempt list 
#         improved_content_temp_list.append (out.content)
#         print (f"batch {i + 1 } content:",out.content)

#     print ("*** Improve Writng complete ***")

#      # Get today's date in the format YYYY-MM-DD
#     today_date = datetime.now().strftime("%Y-%m-%d")

#     # Construct the filename with today's date
#         # Create directory if it doesn't exist
#     dir_name = 'raw_improved_content'
#     if not os.path.exists(dir_name):
#         os.makedirs(dir_name)

#     # Define the path for the output file
#     file_path = os.path.join(dir_name, f'raw_improved_content_{today_date}.txt')

#     # Writing the pain points to the text file
#     with open(file_path, 'w') as file:
#         for idea in improved_content_temp_list:
#             file.write(idea + '\n')  # Write each pain point on a new line

#     print(f"***Improved Painpoints Content saved to '{file_path}' ***")

#     return improved_content_temp_list

In [49]:
improved_content_temp_list
ideas = improve_writing_ol (improved_content_temp_list,main_content_temp_list)

NameError: name 'improve_writing_ol' is not defined

## Ideas Pipline -> with Ideas Agent - > deprecated
IMPORTANT -> the online ideas generation pipeline leads to halucination of ideas, and is deprecated.Save for documentation

Initialize agent function

In [ ]:
# def init_ideas_agent (input_data_batch: list,memory_llm,agent_llm):
# # Define memory for this agent 
#     ideas_memory = ConversationBufferMemory (
#         memory_key="ideas_memory",
#         return_messages=True,
#         llm=memory_llm,
#         max_tokens=1000
#     )
#     ## be careful
#     ideas_memory.clear()

#     empty_system_message_ideas = SystemMessagePromptTemplate.from_template(p_system_message_ideas_offline)

#     system_message_ideas =empty_system_message_ideas.format(input_data=input_data_batch) 

#     agent_kwargs = {
#         "extract_prompt_messages" : [MessagesPlaceholder(variable_name="ideas_memory")],
#         "system_message" : system_message_ideas,
#     }

#     # Create Agent 

#     # initialize agent
#     ideas_agent = initialize_agent (
#         tools=[],
#         llm=agent_llm,
#         agent=AgentType.OPENAI_FUNCTIONS,
#         verbose=True,
#         agent_kwargs=agent_kwargs,
#         memory=ideas_memory,
#     )
#     print ("Ideas Agent Intialized")
#     return ideas_agent

In [ ]:
# ideas_temp_list= []
# ideas = get_ideas (ideas_temp_list, painpoints_temp_list,memory_llm=gpt4,agent_llm=pplx)

In [ ]:
# ## The main function to get ideas 
# def get_ideas (ideas_temp_list:list, painpoints_temp_list: list ,memory_llm = gpt3, agent_llm = pplx):

#     for i, pp_batch in enumerate (painpoints_temp_list):  # need to add to this 
#         count  = 1 
#         # initialize agaent 
#         ideas_agent = init_ideas_agent(pp_batch,memory_llm=memory_llm,agent_llm=agent_llm)
        
#         # TODO: run ai
#         q = """ 
#             FOR EACH PAINPOINTS
#             1. Generate 1 specific saas idea Solution that directly solves the problem
#             2. Generate 3 key features FOR EACH saas ideas generated  Output format :(8-15 word description of each features)
#             3. generate 2,  1 to 2 word tags for this item describing their category or domain 

#             FINAL OUTPUT FORMAT (OUTPUT IN BELOWR FORMAT AND NOTHING ELSE: 
#             painpoint: xxxx  (8 to 12 words)
#             source: xxx (ONLY IF APPLICABLE, IF NOT: "")
#             link: xxx(ONLY IF APPLICABLE, IF NOT: "")
#             description: xxxxxx (3 sentences)  
#             idea: (6-10 word title of the idea; WRITING STYLE: Technical, Prfoessional, Tech Blogger)
#             features: xxx , xxx , xxx (8-15 word description of each features,vary;WRITING STYLE: Technical, Prfoessional, Tech Blogger)
#             tags: xxx, xx 
#             """
#         out = agent.invoke({"input": q}) 
#         print (f"*** Idea Generate Batch {i + 1} Complete! ***")

#         # append to ideas_tempt list 
#         ideas_temp_list.append (out["output"])

#     print ("*** Ideas Generation Complete ***")

#      # Get today's date in the format YYYY-MM-DD
#     today_date = datetime.now().strftime("%Y-%m-%d")

#     # Construct the filename with today's date
#         # Create directory if it doesn't exist
#     dir_name = 'ideas_raw'
#     if not os.path.exists(dir_name):
#         os.makedirs(dir_name)

#     # Define the path for the output file
#     file_path = os.path.join(dir_name, f'ideas_{today_date}.txt')

#     # Writing the pain points to the text file
#     with open(file_path, 'w') as file:
#         for idea in ideas_temp_list:
#             file.write(idea + '\n')  # Write each pain point on a new line

#     print(f"*** Painpoints saved to '{file_path}' ***")

#     return ideas_temp_list
        